# Blokkmark

## Bronse-tabell

### Pre-prosessering

Henter skjema og oppretter tabell

In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import *

In [0]:
catalog_dev = "`land_auto-gen-kart_dev`"
schema_dev = "dl_bildesegmentering"
spark.sql(f"USE CATALOG {catalog_dev}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_dev}")
spark.sql(f"USE SCHEMA {schema_dev}")
bronze_table = "blokkmark_bronze"

In [0]:
q = f"""
CREATE TABLE IF NOT EXISTS {bronze_table} (
    lokalid STRING,
    geometry BINARY,
    oppdateringsdato TIMESTAMP,
    grunnforhold INTEGER,
    arealtype INTEGER,
    ingest_time TIMESTAMP
) USING DELTA
"""

spark.sql(q)

### Funksjoner

In [0]:
def read_from_table() -> DataFrame:
    """
    Henter relevant data om blokkmark og samler det i en data frame.

    Returns:
        DataFrame: Tabellinfo om blokkmark
    """
    df_bronze = (
        spark.read.table("land_ngis.silver_fkbar5.arealressursflate")
        .withColumn("oppdateringsdato", to_timestamp(col("oppdateringsdato")))
        .select("lokalid", "geometry", "oppdateringsdato", "grunnforhold", "arealtype")
        .filter((col("grunnforhold") == 41))
    )
    return df_bronze

In [0]:
def write_delta_table(sdf: DataFrame, mode: str="merge") -> None:
    """
    Oppdaterer eller lager delta-tabell for blokkmark med relevant data fra data frame.

    Args:
        sdf (DataFrame): Data frame med blokkmark-date_add
        mode (str): Bestemmer modus for oppdatering, default: 'merge'
    """
    if mode == "overwrite":
        sdf.write.format("delta").option("mergeSchema", "true").mode(
            mode
        ).saveAsTable(bronze_table)
    else:
        delta_tbl = DeltaTable.forName(spark, bronze_table)

        delta_tbl.alias("target").merge(
            sdf.alias("source"), condition="target.lokalid = source.lokalid"
        ).whenMatchedUpdate(
            condition="target.ingest_time < source.ingest_time",
            set={col: f"source.{col}" for col in sdf.columns},
        ).whenNotMatchedInsert(
            values={col: f"source.{col}" for col in sdf.columns}
        ).execute()

### Program

In [0]:
def main():
    df = read_from_table()
    df = df.withColumn("ingest_time", current_timestamp())
    write_delta_table(df)

In [0]:
if __name__ == "__main__":
    main()